In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
torch.cuda.empty_cache()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM



phi_tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)
phi_model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct", trust_remote_code=True)

# # Load model directly
# from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("anupam413/phi2-finetuned-lora", trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained("anupam413/phi2-finetuned-lora", trust_remote_code=True)

phi_tokenizer.pad_token = phi_tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [4]:
DATA = [
    {
        "outgoing_trait": [
            "stay-home"
        ],
        "occupation": [
            "unemployed"
        ],
        "frequent_visits": [],
        "regular_activity": [
            "gym"
        ],
        "likes": [
            "pets",
            "cars"
        ],
        "app_usages": [
            "utility",
            "entertainment"
        ]
    }]

In [5]:
import random
import json

# Your list of JSON objects
user_behavior_data = DATA

# Select a random JSON object from the list
selected_json = random.choice(user_behavior_data)

# Select a random key from the selected JSON object
random_key = random.choice(list(selected_json.keys()))

# Access the data corresponding to the random key
random_key_data = selected_json.get(random_key, [])

json_data = {random_key: random_key_data}

# Convert the dictionary to a JSON string
user_behavior = json.dumps(json_data, indent=4)

print("Randomly selected key:", random_key)
print("Data for the randomly selected key:", random_key_data)
print("Behavior:", user_behavior)

Randomly selected key: likes
Data for the randomly selected key: ['pets', 'cars']
Behavior: {
    "likes": [
        "pets",
        "cars"
    ]
}


In [6]:
# few_shot_followup_prompt = """
# [SYSTEM]
# You are a personal assistant responsible for generating a conversational and engaging follow-up question to the initial question as the user answered No.
# Your task is to ask for behvior based on their current mood as the user has denied the current generated behavior.
# Keep conversation friendly, short, clear, and concise. 
# Avoid unnecessary introductions and answer the user's questions directly. 
# Respond in a normal, conversational manner while being friendly and helpful.
# The answer should be a single question and nothing else.

# Example 1:
# Behavior: "app_usages": ["music"],
# Current mood: happy,
# Initial question: "Do you really enjoy music or just pretending to be cool? Yes/No"
# Answer: "No"
# Follow-up question: "What kind of apps do you usually use to keep that happy vibe going?"

# Example 2:
# Behavior: "frequent_visits": ["landmarks","nature"],
# Current mood: sad,
# Initial question: "Are you visiting landmarks and nature spots to find happiness? Yes/No"
# Answer: "No"
# Follow-up question: "If landmarks and nature aren't your thing right now, where do you prefer to go to lift your spirits?"

# Example 3:
# Behavior: "likes": ["pets"],
# Current mood: angry,
# Initial question: "Do you like pets to calm down or to train them to take over the world? Yes/No"
# Answer: "No"
# Follow-up question: "So pets aren't your style at the moment. What do you find helps to ease your anger?"

# [USER]
# """''

In [7]:
behaviors = [
     {"outgoing_trait": [
        {
            "outgoing": True,
            "verified": False
        }
    ]},
    {"trait": [
        {
            "outgoing": True,
            "verified": False
        }
    ]},
    {"occupation": [
        {
            "unemployed": True,
            "verified": False
        }
    ]},
    {"frequent_visits": [
        {
            "nature": True,
            "verified": False
        }
    ]},
    {"regular_activity": [
        {
            "gym": True,
            "verified": False
        }
    ]},
    {"likes": [
        {
            "bikes": True,
            "verified": False
        },
        {
            "cars": True,
            "verified": False
        }
    ]},
    {"app_usages": [
        {
            "music": True,
            "verified": False
        }
    ]}
]

In [8]:
phi_model = phi_model.to(device)

In [9]:
current_mood = "happy"

In [53]:
import re

def generate_question(current_mood, initial_question):
    few_shot_followup_prompt = f"""
    [SYSTEM]
    You are a personal assistant responsible for generating a normal, conversational and engaging follow-up question to the initial question as the user answered No.
    Your task is to generate question suitable for their current mood.
    Keep conversation friendly, short, clear, and concise. 
    Avoid unnecessary introductions and answer the user's questions directly. 
    You should give the follow-up question only and nothing else.
    """
    user_content = "[USER]" + f"Is in {current_mood} mood right now. At first, the {initial_question} was asked." + "[ANSWER]: No"
    prompt = few_shot_followup_prompt + user_content + "[FOLLOW UP QUESTION]:"
#     prompt = f"{few_shot_followup_prompt}\nBehavior: {behavior_str}\nCurrent mood: {current_mood}\nInitial question:{initial_question}\n[Follow-up question if the answer is \"No\"]: "
#     prompt = few_shot_followup_prompt.format(
#         behavior=behavior_str, 
#         current_mood=current_mood, 
#         initial_question=initial_question
#     )
    inputs = phi_tokenizer(prompt, return_tensors="pt").to(device)
    outputs = phi_model.generate(inputs["input_ids"], max_length=200, temperature=0.3, top_p=0.9, num_return_sequences=1, do_sample=True)
    question = phi_tokenizer.decode(outputs[0], skip_special_tokens=True)
    
#     match = re.search(r'\[FOLLOW UP QUESTION\]: (.+)', question)
#     match = match.group(1).strip()
        
    return question

In [54]:
# Function to extract and format the data
def extract_and_format_data(data):
    extracted_data = []
    for item in data:
        for key, values in item.items():
            for value in values:
                for sub_key in value:
                    if sub_key != "verified":
                        extracted_data.append({key: sub_key})
    return extracted_data

In [55]:
behaviors

[{'outgoing_trait': [{'outgoing': True, 'verified': False}]},
 {'trait': [{'outgoing': True, 'verified': False}]},
 {'occupation': [{'unemployed': True, 'verified': False}]},
 {'frequent_visits': [{'nature': True, 'verified': False}]},
 {'regular_activity': [{'gym': True, 'verified': False}]},
 {'likes': [{'bikes': True, 'verified': False},
   {'cars': True, 'verified': False}]},
 {'app_usages': [{'music': True, 'verified': False}]}]

In [56]:
formatted_data = extract_and_format_data(behaviors)

In [57]:
initial_question = [
    "Do you enjoy socializing because you're outgoing? Yes/No",
    "Do you feel happy because you consider yourself outgoing? Yes/No",
    "Do you find joy in being unemployed? Yes/No",
    "Do you visit nature spots to stay happy? Yes/No",
    "Do you go to the gym to maintain your happiness? Yes/No",
    "Do you like bikes to keep your mood uplifted? Yes/No",
    "Do you enjoy cars to keep yourself happy? Yes/No",
    "Do you use music apps to stay happy? Yes/No"
]

In [58]:
formatted_data

[{'outgoing_trait': 'outgoing'},
 {'trait': 'outgoing'},
 {'occupation': 'unemployed'},
 {'frequent_visits': 'nature'},
 {'regular_activity': 'gym'},
 {'likes': 'bikes'},
 {'likes': 'cars'},
 {'app_usages': 'music'}]

In [59]:
import time

# Generate questions for each behavior
for i, behavior_data in enumerate(formatted_data):
    for key, value in behavior_data.items():
        print({key:value})
        t1 = time.time()
        question = generate_question(current_mood , initial_question[i])
        print("**"*100)
        print("initialQuestion:", initial_question[i])
        print(f"Generated question for {key}: {question}")
        print("Time taken:", time.time()-t1)
        print("--"*100)

{'outgoing_trait': 'outgoing'}
********************************************************************************************************************************************************************************************************
initialQuestion: Do you enjoy socializing because you're outgoing? Yes/No
Generated question for outgoing_trait: 
    [SYSTEM]
    You are a personal assistant responsible for generating a normal, conversational and engaging follow-up question to the initial question as the user answered No.
    Your task is to generate question suitable for their current mood.
    Keep conversation friendly, short, clear, and concise. 
    Avoid unnecessary introductions and answer the user's questions directly. 
    You should give the follow-up question only and nothing else.
    [USER]Is in happy mood right now. At first, the Do you enjoy socializing because you're outgoing? Yes/No was asked.[ANSWER]: No[FOLLOW UP QUESTION]: What activities do you prefer when you're not 

## Test

In [60]:
text = """[SYSTEM]
    You are a personal assistant responsible for generating a normal, conversational and engaging follow-up question to the initial question as the user answered No.
    Your task is to generate question suitable for their current mood.
    Keep conversation friendly, short, clear, and concise. 
    Avoid unnecessary introductions and answer the user's questions directly. 
    You should give the follow-up question only and nothing else.
    [USER]Is in happy mood right now. At first, the Do you use music apps to stay happy? Yes/No was asked.[ANSWER]: No[FOLLOW UP QUESTION]: What other activities do you enjoy that help you maintain a positive mood?
    [USER]I'm feeling quite stressed lately. Do you think meditation helps with stress? Yes/No was asked.[ANSWER]: Yes"""

In [62]:
match = re.search(r'\[FOLLOW UP QUESTION\]: (.+)', question)
match = match.group(1).strip()

In [63]:
match

'What other activities do you enjoy that help you maintain a positive mood?'